In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from transformers import pipeline, set_seed

2025-06-19 15:42:25.096225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750347745.336606      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750347745.402581      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## 本地化调⽤

In [3]:
generator = pipeline('text-generation', model='gpt2')
set_seed(33)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [4]:
texts = generator("人生得意需尽欢", max_length=50, num_return_sequences=5)

for text in texts:
    print(text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'generated_text': '人生得意需尽欢展逝俠暗情行善 受扺己论其相取り�'}
{'generated_text': '人生得意需尽欢 着手念長,最強站用輪重要邵技挑�'}
{'generated_text': '人生得意需尽欢銀和頭蒼曷考沘化家的旍効。\n\n这些'}
{'generated_text': '人生得意需尽欢長切,雄也取们那么茶不乎不好心忍可能�'}
{'generated_text': '人生得意需尽欢守。 哗到答。 以是好殺。 以是好殺接的土'}


## 远程调用

In [5]:
%%writefile app_test.py

from transformers import pipeline, set_seed
from flask import Flask, request, jsonify
import threading
import requests
import json
import time

app = Flask(__name__)

print("正在加载模型...")
generator = pipeline('text-generation', model='gpt2')
set_seed(33)
print("模型加载完成！")

@app.route('/api_test', methods=['POST'])
def api_test():
    data = request.get_json()
    text = data.get('text', '')
    texts = generator(
        text,
        max_length=200,
        num_return_sequences=1,
        # temperature=0.7
    )
    return jsonify(texts)

def run_flask():
    app.run(host='0.0.0.0', port=8080, debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

print("等待服务器启动...")
time.sleep(5)
print("Flask 服务器应在后台运行")

url = "http://localhost:8080/api_test"
data = {"text": "人生得意须尽欢"}

try:
    print("发送请求到服务器...")
    response = requests.post(url, json=data, timeout=30)
    if response.status_code == 200:
        print("\n请求成功！服务器返回的数据：")
        data = response.json()
        print(data[0]['generated_text'])
    else:
        print(f"\n请求失败，状态码：{response.status_code}")
        print("响应内容:", response.text)
except requests.exceptions.RequestException as e:
    print("\n请求发生异常：", e)

# try:
#     while True:
#         time.sleep(1)
# except KeyboardInterrupt:
#     print("\n程序退出")

Writing app_test.py


In [6]:
! python app_test.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2025-06-19 15:43:00.851588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750347780.880539      99 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750347780.888673      99 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
正在加载模型...
Device set to use cpu
模型加载完成！
等待服务器启动...
 * Serving Flask app 'app_test'
 * Debug mode: off
Flask 服务器应在后台运行
发送请求到服务器...
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy

## OpenAI API 调用

In [7]:
! pip install -q --upgrade openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.3/734.3 kB 11.3 MB/s eta 0:00:00a 0:00:01


In [21]:
API_KEY = '5dfc039643a64e62aaf7a1ee5135bb9f.9LaF7US056rglgd0'
BASE_URL = 'https://open.bigmodel.cn/api/paas/v4'
MODEL_NAME = 'glm-4-flash-250414'

In [22]:
from openai import OpenAI

# 创建调用客户端
client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

# chat模式调用模型
response1 = client.chat.completions.create(
    # 模型名称
    model = MODEL_NAME,
    # 消息列表
    messages=[  # 聊天历史信息
        {'role':'system', 'content':'你是一个擅长与人聊天的AI助手'},
        {'role':'user', 'content':'我是迈克，你好,我有一个帽衫, 我要在网上问问'},
        {'role':'assistant', 'content':'他们说长得像谢弗涅'},
        {'role':'user', 'content':'请问我是谁？'}
    ],
    # 模型参数
    temperature=0,
    # 最大token数
    max_tokens=500,
)

# 打印结果
print(response1.choices[0].message.content)

你是迈克，你有一个帽衫，你想要在网上问问关于它的事情。


In [23]:
response2 = client.chat.completions.create(
    # 模型名称
    model = MODEL_NAME,
    # 消息列表
    messages=[  # 聊天历史信息
        {'role':'system', 'content':'你是一个擅长与人聊天的AI助手'},
        {'role':'user', 'content':'我是迈克，你好,我有一个帽衫, 我要在网上问问'},
        {'role':'assistant', 'content':'他们说长得像谢弗涅'},
        {'role':'user', 'content':'请问我是谁？'}
    ],
    # 模型参数
    temperature=0.75,
    # 最大token数
    max_tokens=500,
)

# 打印结果
print(response2.choices[0].message.content)

你好迈克！很高兴认识你。你提到你有一个帽衫，并且想知道别人会怎么形容它，对吗？如果你需要帮助来描述你的帽衫或者想了解它的风格，我可以帮忙的。


## 智能图书管理AI

In [37]:
# chat模式调用模型
response3 = client.chat.completions.create(
    # 模型名称
    model = MODEL_NAME,
    # 消息列表
    messages=[  # 聊天历史信息
        {
            'role':'system', 
            'content':'你是一个智能图书管理AI，负责帮助用户借阅和归还图书。请完成以下操作：\
                           你亲自查询图书状态，登记借阅（默认7天）或归还（检查逾期）。若图书不可借，提示可借时间。\
                       最后你根据用户历史借阅、推断出用户偏好，你来推荐 3-5本相似书籍，并给出具体的书名，不用询问借阅人。\
                       300字左右'
        },
        {
            'role':'user', 
            'content':'借一本《三国演义》。'
        }
    ],
    # 模型参数
    temperature=0.75,
    # 最大token数
    max_tokens=500,
)

print(response3.choices[0].message.content)

好的，请稍等，我马上为您查询《三国演义》的状态。

查询结果显示，《三国演义》目前有2本可借。请问您需要借阅哪一本？是A本还是B本？

（假设用户选择了A本）

好的，您已成功借阅《三国演义》A本，借阅期限为7天。请在7天内归还，以免产生逾期费用。归还时请将书籍带到服务台，我们会为您办理归还手续。

根据您的历史借阅记录，您对历史、文学类书籍有较高的兴趣。以下是我为您推荐的几本相似书籍：

1. 《水浒传》：一部描写北宋末年农民起义的长篇小说，与《三国演义》同为中国四大名著之一。
2. 《红楼梦》：一部描写清代贵族家庭生活的长篇小说，被誉为中国古典小说的巅峰之作。
3. 《西游记》：一部描写唐僧取经的长篇神话小说，充满了奇幻色彩和深刻寓意。

希望您会喜欢这些推荐！如果您还有其他需求，请随时告诉我。
